# 동적 크롤링 (2)
- 동적 크롤링과 파이썬의 'csv' 라이브러리를 함께 사용

## csv 파일에 단어와 그 뜻을 입력

In [2]:
from selenium import webdriver
import time
import csv

# 자동화된 크롬 창 실행
driver = webdriver.Chrome('./chromedriver')

# 파파고 웹 페이지 접속
papago_url = 'https://papago.naver.com/'
driver.get(papago_url)

# 시간적 여유 3초
time.sleep(3)

# 작성할 'my_papago.csv' 파일을 생성하여 변수 'f'에 저장
f = open('./my_papago.csv', 'w', newline = '')

# CSV 파일을 작성하는 객체 변수 'wtr' 생성
wtr = csv.writer(f)

# 열 제목 작성
wtr.writerow(['영단어', '번역결과'])

# 그만 두고 싶을 떄 까지 영단어 입력, 저장 반복
while True:
    keyword = input('번역할 영단어 입력 (0 입력하면 종료) : ')
    if keyword == '0':
        print('번역 종료')
        break

    # 영단어 입력, 번역 버튼 클릭
    form = driver.find_element_by_css_selector('textarea#txtSource')
    form.send_keys(keyword)

    button = driver.find_element_by_css_selector('button#btnTranslate')
    button.click()
    time.sleep(1)

    # 번역 결과 저장
    output = driver.find_element_by_css_selector('div#txtTarget').text
    
    # my_papago.csv 파일에 [영단어, 번역결과] 작성
    wtr.writerow([keyword, output])
    
    # 영단어 입력 칸 초기화  # [ Selenium의 HTML 요소 ].clear()
    driver.find_element_by_css_selector('textarea#txtSource').clear()

# 크롬 창 닫기
driver.close()

# 파일 닫기
f.close()

번역할 영단어 입력 (0 입력하면 종료) : apple
번역할 영단어 입력 (0 입력하면 종료) : work
번역할 영단어 입력 (0 입력하면 종료) : science
번역할 영단어 입력 (0 입력하면 종료) : 0
번역 종료


## 사전 속 중복 없애기
- csv 파일에 있는 영단어일 경우, 저장하지 않음
- csv 파일에 없는 영단어는 번역 결과 저장

In [7]:
from selenium import webdriver
import time
import csv

driver = webdriver.Chrome('./chromedriver')
papago_url = 'https://papago.naver.com/'
driver.get(papago_url)
time.sleep(3)

# 읽어올 'my_papago.csv' 파일을 변수 'f'에 저장
f = open('./my_papago.csv', 'r') # CSV 파일을 불러오기 때문에 읽기 옵션 'r'

# CSV 파일의 모든 데이터를 변수 'rdr'에 저장
rdr = csv.reader(f)

# 'rdr'의 첫 번째 (열 제목)는 건너뜀
next(rdr)

# 딕셔너리 생성
my_dict = {}

# 딕셔너리에 영단어와 번역 결과를 모두 저장
for row in rdr:
    keyword = row[0]
    korean = row[1]
    my_dict[keyword] = korean
# 파일 닫기
f.close()

# 추가 옵션 'a'를 사용하여 파일 다시 열기
f = open('./my_papago.csv', 'a', newline = '')
wtr = csv.writer(f)

# 단어 입력과 저장을 위한 반복 
while True:
    keyword = input('번역할 영단어 입력 (0 입력하면 종료) : ')
    if keyword == '0':
        print('번역 종료')
        break
    
    # 영단어가 'my_dict'의 키 값 중에 있다면, 이 사실을 알려주고 저장되어있던 번역 결과 출력
    if keyword in my_dict.keys():
        print('이미 번역한 영단어입니다! 뜻은', my_dict[keyword], '입니다.')
    # 위의 경우에 포함되지 않으면, 딕셔너리와 CSV 파일에 추가
    else:
        driver.find_element_by_css_selector('textarea#txtSource').send_keys(keyword)
        driver.find_element_by_css_selector('button#btnTranslate').click()
        time.sleep(1)

        output = driver.find_element_by_css_selector('div#txtTarget').text

        # CSV 파일에 행 추가
        wtr.writerow([keyword, output])

        # 딕셔너리에 추가
        my_dict[keyword] = output

        driver.find_element_by_css_selector('textarea#txtSource').clear()
        
# 크롬 창 닫기
driver.close()

# 파일 닫기
f.close()

print(f"나의 사전 : {my_dict}")

번역할 영단어 입력 (0 입력하면 종료) : song
번역할 영단어 입력 (0 입력하면 종료) : knee
번역할 영단어 입력 (0 입력하면 종료) : 0
번역 종료
나의 사전 : {'apple': '사과', 'work': '일하다.', 'science': '과학', 'yellow': '노란 색', 'aplle': '아플', 'home': '집입니다', 'song': '노래를 부르다', 'knee': '무릎을 꿇다'}


## Quiz
- 번역 결과가 제대로 된 번역인지 확인
한글 뜻을 입력하고 영어를 출력

In [13]:
from selenium import webdriver
import time
import csv

driver = webdriver.Chrome('./chromedriver')
papago_url = 'https://papago.naver.com/'
driver.get(papago_url)
time.sleep(3)

# 읽어올 'my_papago.csv' 파일을 변수 'f'에 저장
f = open('./my_papago.csv', 'r') # CSV 파일을 불러오기 때문에 읽기 옵션 'r'

rdr = csv.reader(f)

next(rdr) # 열의 제목은 필요하지 않는다면 next 함수 사용
# csv 파일의 한국어 부분만 리스트에 담음
korean = []
for row in rdr:
    korean.append(row[1])
    
# 파일 닫기
f.close()
# 영어' ↔ '한국어' 버튼 누르기
driver.find_element_by_css_selector('button.btn_switch___x4Tcl').click()

# 단어 입력과 저장을 위한 반복 
for kor in korean:
    driver.find_element_by_css_selector('textarea#txtSource').send_keys(kor)
    driver.find_element_by_css_selector('button#btnTranslate').click()
    time.sleep(1)

    output = driver.find_element_by_css_selector('div#txtTarget').text
    driver.find_element_by_css_selector('textarea#txtSource').clear()
    print(f"{kor} : {output}")
        
# 크롬 창 닫기
driver.close()

사과 : apple
일하다. : Work.
과학 : Science
노란 색 : Yellow
아플 : It hurts.
집입니다 : Home.
노래를 부르다 : Sing a song.
무릎을 꿇다 : Go down on one's knees.
